# Overview
In this notebook, we will learn about state in LangGraph

State in LangGraph is a way to maintain and track information as an AI system processes data. Think of it as the "system’s memory", allowing it to remember and update information as it moves through different stages of a workflow, or graph.

### Basic State
-  TypedDict means its a python module to define dictionaries with specific types for their keys and values. It's particularly useful when you want to create dictionary-like classes where you know the exact keys and their corresponding value types at runtime.
- Below state can be incredibly useful in many scenarios, such as:
    - Tracking the number of turns in a conversation
    - Counting the occurrences of a specific event
    - Maintaining a simple score, metric, or AI output

In [11]:
from typing import TypedDict

class BasicState(TypedDict):
    count: int

### Complex State
-  This structure is particularly useful for:
    - Chatbots that need to remember conversation history
    - AI assistants that maintain context over multiple interactions
    - Systems that need to track both quantitative (count) and qualitative (messages) data

In [8]:
from typing import TypedDict, Annotated
from langchain_core.messages import HumanMessage, AIMessage

class ComplexState(TypedDict):
    count: int
    messages: Annotated[list[HumanMessage | AIMessage], "conversation_history"]


### State Modification Functions
Once we have our state structures defined, we need ways to modify them. In LangGraph, we typically "create new state objects rather than modifying existing ones", adhering to the principles of immutability

Following functions demonstrate how we can:
- Increment a counter in our basic state
- Add new messages to our complex state, distinguishing between human and AI messages
- Create new state objects that reflect these changes

In [10]:
def increment_count(state: BasicState) -> BasicState:
    return BasicState(count=state["count"] + 1)

def add_message(state: ComplexState, message: str, is_human: bool = True) -> ComplexState:
    new_message = HumanMessage(content=message) if is_human else AIMessage(content=message)
    return ComplexState(
        count=state["count"],
        messages=state["messages"] + [new_message]
    )

### Simple Graphs with States

In [20]:
from langgraph.graph import StateGraph, END

def create_simple_graph():
    workflow = StateGraph(BasicState)
    
    def increment_node(state: BasicState):
        return {"count": state["count"] + 1}
    
    workflow.add_node("increment", increment_node)
    workflow.set_entry_point("increment")
    workflow.add_edge("increment", END)
    
    return workflow.compile()


### Complex graph with States

In [17]:
def create_complex_graph():
    workflow = StateGraph(ComplexState)
    
    def process_message(state: ComplexState):
        last_message = state["messages"][-1].content if state["messages"] else "No messages yet"
        response = f"Received: {last_message}. Count is now {state['count'] + 1}"
        return {
            "count": state["count"] + 1,
            "messages": state["messages"] + [AIMessage(content=response)]
        }
    
    workflow.add_node("process", process_message)
    workflow.set_entry_point("process")
    workflow.add_edge("process", END)
    
    return workflow.compile()

### Execution

In [21]:

# Interactive Session
def run_interactive_session():
    print("Welcome to the Interactive LangGraph State Lesson!")
    
    print("\nStep 1: Basic State")
    basic_state = BasicState(count=0)
    print(f"Initial basic state: {basic_state}")
    
    print("\nStep 2: More Complex State")
    complex_state = ComplexState(count=0, messages=[])
    print(f"Initial complex state: {complex_state}")
    
    print("\nStep 3: State Modification")
    modified_basic = increment_count(basic_state)
    print(f"Modified basic state: {modified_basic}")
    
    modified_complex = add_message(complex_state, "Hello, LangGraph!")
    print(f"Modified complex state: {modified_complex}")
    
    print("\nStep 4: Simple Graph with State")
    simple_graph = create_simple_graph()
    result = simple_graph.invoke(BasicState(count=0))
    print(f"Simple graph result: {result}")
    
    print("\nStep 5: Complex Graph with State")
    complex_graph = create_complex_graph()
    initial_state = ComplexState(count=0, messages=[HumanMessage(content="Hello, LangGraph!")])
    result = complex_graph.invoke(initial_state)
    print(f"Complex graph result: {result}")

In [22]:
run_interactive_session()

Welcome to the Interactive LangGraph State Lesson!

Step 1: Basic State
Initial basic state: {'count': 0}

Step 2: More Complex State
Initial complex state: {'count': 0, 'messages': []}

Step 3: State Modification
Modified basic state: {'count': 1}
Modified complex state: {'count': 0, 'messages': [HumanMessage(content='Hello, LangGraph!')]}

Step 4: Simple Graph with State
Simple graph result: {'count': 1}

Step 5: Complex Graph with State
Complex graph result: {'count': 1, 'messages': [HumanMessage(content='Hello, LangGraph!'), AIMessage(content='Received: Hello, LangGraph!. Count is now 1')]}
